In [13]:
# required libraries
from faker import Faker
import random
from pymongo import MongoClient
import json


In [14]:
# generate fake sales data using faker library

fake = Faker()

def generate_fake_sales_data(num_records):
    data = []
    for _ in range(num_records):
        record = {
            'customer_name': fake.name(),
            'product_name': fake.word(),
            'quantity': random.randint(1, 100),
            'price': round(random.uniform(10, 100), 2)
        }
        data.append(record)
    return data

In [15]:
#load data into mangodb sever

def load_data_to_mongodb(data):
    client = MongoClient('mongodb://localhost:27017/')
    db = client['sales_db']
    collection = db['sales_collection']
    collection.insert_many(data)

In [16]:
# now extract fake data from the mongo db

def extract_details_from_mongodb():
    client = MongoClient('mongodb://localhost:27017/')
    db = client['sales_db']
    collection = db['sales_collection']

    pipeline = [
        {
            '$group': {
                '_id': '$customer_name',
                'total_quantity': {'$sum': '$quantity'},
                'total_price': {'$sum': {'$multiply': ['$quantity', '$price']}}
            }
        },
        {
            '$sort': {'total_price': -1}
        }
    ]

    result = collection.aggregate(pipeline)

    # Print the extracted details
    for doc in result:
        print(doc)

In [17]:
#create a pipeline function to work

def etl_pipeline(num_records):
    # Generate fake sales data
    fake_sales_data = generate_fake_sales_data(num_records)

    # Load data into MongoDB
    load_data_to_mongodb(fake_sales_data)

    # Extract required details from MongoDB
    extract_details_from_mongodb()

# Run the ETL pipeline with 1000 records
etl_pipeline(1000)


{'_id': 'Michelle Rodriguez', 'total_quantity': 187, 'total_price': 13797.349999999999}
{'_id': 'Katherine Lopez', 'total_quantity': 180, 'total_price': 9783.0}
{'_id': 'Christopher Smith', 'total_quantity': 97, 'total_price': 9646.65}
{'_id': 'Adrian Cooper', 'total_quantity': 98, 'total_price': 9261.0}
{'_id': 'Tyler Moreno', 'total_quantity': 95, 'total_price': 8950.9}
{'_id': 'Kaitlyn Lewis', 'total_quantity': 92, 'total_price': 8949.76}
{'_id': 'Erika Sanchez', 'total_quantity': 91, 'total_price': 8848.84}
{'_id': 'Lindsey Solomon', 'total_quantity': 98, 'total_price': 8798.44}
{'_id': 'Jacqueline Nguyen', 'total_quantity': 99, 'total_price': 8726.85}
{'_id': 'Joel Wilson', 'total_quantity': 97, 'total_price': 8688.289999999999}
{'_id': 'Kristin Vaughn', 'total_quantity': 98, 'total_price': 8669.08}
{'_id': 'Wanda Bell', 'total_quantity': 96, 'total_price': 8656.32}
{'_id': 'Steven Patterson', 'total_quantity': 131, 'total_price': 8430.329999999998}
{'_id': 'Ruth Chavez', 'total_q